In [1]:
%env BASE_DIR=/home/jupyter/fs_shopify

env: BASE_DIR=/home/jupyter/fs_shopify


In [2]:
!gsutil cp gs://cs327e-open-access-2/shopify.zip $BASE_DIR

Copying gs://cs327e-open-access-2/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [ ]:
!unzip $BASE_DIR/shopify.zip

Archive:  /home/jupyter/fs_shopify/shopify.zip
replace __MACOSX/._shopify? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows:
    print(row)

## 1. Creating Category Collection - Access Pattern #1

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
apps_cat_rows = df.values.tolist()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat_rows = df.values.tolist()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

# main collection Category
for cat_row in cat_rows:
    cat_record = {}
    current_cat = cat_row[0]
    cat_record['cat_id'] = cat_row[0]
    cat_record['title'] = cat_row[1]

    cat_ref = db.collection('Category').document(cat_row[1])
    batch.set(cat_ref, cat_record)
    batch.commit()

    # for each cat_id/title finding suitable app_ids
    for apps_cat_row in  apps_cat_rows:
        if apps_cat_row[1] == current_cat: #cat_row[0]
            #appid = app_cat_row[0]
        #creating a dict (doc) for the matching app_id
            for apps_row in apps_rows:
                if apps_row[0] == apps_cat_row[0]:
                    app_record = {}
                    app_record['app_id'] = apps_row[0]
                    app_record['url'] = apps_row[1]
                    app_record['app_title'] = apps_row[2]
                    app_record['developer'] = apps_row[3]
                    app_record['developer_link'] = apps_row[4]
                    app_record['icon'] = apps_row[5]
                    app_record['rating'] = apps_row[6]
                    app_record['reviews_count'] = apps_row[7]

                    app_ref = cat_ref.collection('Apps').document(apps_row[0])
                    batch.set(app_ref, app_record)
                    batch.commit()
                    break
                    

## 2. Queries related to Access Pattern #1

In [2]:
cat_ref = db.collection('Category')
query = cat_ref.where('title', '==', 'Reporting')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

Reporting => {'cat_id': 'c576a841fd4f333a6f074d68e76a1d37', 'title': 'Reporting'}


#### 2a. Counts of documents in collections and subcollections (Access Pattern #1).
*Notice, # of apps in Apps subcollection is not the same as in Apps collection bc apps in the subcollection are not unique, may be included in several categories*

In [3]:
count_Category = db.collection_group('Category')
results = count_Category.stream()
n = 0
for line in results:
    sub = db.collection('Category').document('Apps')
    #print(f'{line.id} => {line.to_dict()}')
    n += 1
print('Category collection:', n)


Category collection: 12


In [4]:
n = 0
cat_ref = db.collection('Category')
main_coll = cat_ref.stream()
for el in main_coll:
    k = 0
    subs = cat_ref.document(el.id).collection('Apps').stream()
    for sub in subs:
        n += 1
        k += 1
    print(f'Category "{el.id}" Apps number:', k)    
print('Apps subcollection Total:', n)

Category "Customer support" Apps number: 310
Category "Finances" Apps number: 133
Category "Finding and adding products" Apps number: 176
Category "Inventory management" Apps number: 251
Category "Marketing" Apps number: 794
Category "Orders and shipping" Apps number: 602
Category "Places to sell" Apps number: 83
Category "Productivity" Apps number: 190
Category "Reporting" Apps number: 249
Category "Sales and conversion optimization" Apps number: 957
Category "Store design" Apps number: 1537
Category "Trust and security" Apps number: 101
Apps subcollection Total: 5383


In [13]:
cat_ref = db.collection('Category')
query = cat_ref
results = query.stream()
for result in results:
    cat = result.to_dict()
    print(f'{result.id} =>', '{}'.format(cat['cat_id']))

Customer support => 64430ad2835be8ad60c59e7d44e4b0b1
Finances => 26a72de0d02e0e4e5f615332d61a878e
Finding and adding products => bd125b49ce6b0425d29e33cb8b49a496
Inventory management => 9fe78acd078fc030de72c896adc93a2d
Marketing => c769c2bd15500dd906102d9be97fdceb
Orders and shipping => 30ea2315e910c5e5335de42d5e87dca5
Places to sell => 30a930262efca129caafcd586bc7e6fe
Productivity => 737ad50051083aa051d127a53b3ac0da
Reporting => c576a841fd4f333a6f074d68e76a1d37
Sales and conversion optimization => 5eb4e29e50e3f178acc614236ed107f4
Store design => c3f9db73c29bc33607778f9935c3c4dc
Trust and security => e44cd5379a3166568741a07cc81ef6e5


#### 2b. Top 10 Productivity apps by rating in descending order.

In [14]:
cat_ref = db.collection('Category')
query = cat_ref.document('Productivity').collection('Apps').order_by('rating', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
print('Top 10 Productivity Category Apps:')
for result in results:
    sub = result.to_dict()
    print(f'ID: {result.id} =>', 'title: "{}", developer: {}, rating: {}, reviews count: {}'.\
          format(sub['app_title'], sub['developer'], sub['rating'], sub['reviews_count']))
    

Top 10 Productivity Category Apps:
ID: fe5ae45c-379c-42bf-be7d-b5f6e15fc13c => title: "Customer Tags", developer: Union Works Apps, rating: 5.0, reviews count: 21
ID: f99bb1e3-f326-4f10-8901-491652e9809b => title: "Order Tagger", developer: Union Works Apps, rating: 5.0, reviews count: 68
ID: f864e3bd-da0e-41dc-be65-984325331475 => title: "SilkRoad ‑ Facebook Auto Ads", developer: SilkRoad, rating: 5.0, reviews count: 2
ID: f5344e64-9cda-4d97-b198-2aaeb5170518 => title: "Xporter Data Export Tool", developer: Modd Apps Inc., rating: 5.0, reviews count: 223
ID: ed77a32d-0fa3-458b-b639-e01ea7b78ec0 => title: "FraudBlock Fraud Prevention", developer: ShopFox, rating: 5.0, reviews count: 6
ID: e0e231d6-4988-4a8c-ad84-cf18d0f38738 => title: "Automation Fox", developer: Automation Fox, rating: 5.0, reviews count: 1
ID: d75fa395-3a4d-41c3-88d1-2f802c1e2411 => title: "AirPower", developer: BaseGenius, rating: 5.0, reviews count: 1
ID: d0be03ff-f74d-4fb5-8d2c-79541a1aea1b => title: "Excelify", d

## 3. Creating Apps Collection - Access Pattern #2,3,4

In [5]:
## Olga's Copy
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
price_plans = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
price_feats = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_bens = df.values.tolist()

# main collection Apps
for apps_row in apps_rows:
    app_record = {}
    app_record['app_id'] = apps_row[0]
    app_record['url'] = apps_row[1]
    app_record['app_title'] = apps_row[2]
    app_record['developer'] = apps_row[3]
    app_record['developer_link'] = apps_row[4]
    app_record['icon'] = apps_row[5]
    app_record['rating'] = apps_row[6]
    app_record['reviews_count'] = apps_row[7]

    app_ref = db.collection('Apps').document(apps_row[0])
    batch.set(app_ref, app_record)
    batch.commit()

    #For each app adding the key benefits
    for key_ben in key_bens:
        if key_ben[0] == apps_row[0]:
            key_record = {}
            key_record['title'] = key_ben[1]
            key_record['description'] = key_ben[2]

            key_ref = app_ref.collection('Key_Benefits').document()
            batch.set(key_ref, key_record)
            batch.commit()
    
    #For each app adding the pricing plans
    for price_plan in price_plans:
        if price_plan[1] == apps_row[0]:           
            price_record = {}
            price_record['plan_id'] = price_plan[0]
            price_record['title'] = price_plan[2]
            price_record['price'] = price_plan[3]

            price_ref = app_ref.collection('Pricing_Plans').document(price_plan[0])
            batch.set(price_ref, price_record)
            batch.commit()
            
            #For each pricing plan adding the features
            for feat in price_feats:
                if feat[0] == price_plan[0]:
                    price_feat = {}
                    price_feat['feature'] = feat[2]

                    feat_ref = price_ref.collection('Features').document()
                    batch.set(feat_ref, price_feat)
                    batch.commit()
                    
            

## 4. Queries related to Access Pattern #2, 3, 4

In [6]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

app_ref = db.collection('Apps')
query = app_ref.where('app_title', '==', 'Live Chat for Slack')
results = query.stream()

for result in results:
    app = result.to_dict()
    print(f'{result.id} =>', '{}, rating: {}'.format(app['app_title'], app['rating']) )


c187acf3-1104-4620-aa00-ca9506ad7ce5 => Live Chat for Slack, rating: 4.2


#### 4a. Counts of documents in collections and subcollections (Access Pattern #2, 3, 4).

In [ ]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

# commented lines are counting and printing pricing plans per each app. Does not make sense printing as there are 3K+ apps
# same approach can be used for other sub-subcollections.
apps_num = 0
n = 0
m = 0
p = 0
app_ref = db.collection('Apps')
main_coll = app_ref.stream()
for el in main_coll:
    apps_num += 1
    #k = 0
    prices = app_ref.document(el.id).collection('Pricing_Plans').stream()
    for price in prices:
        n += 1        
        #k += 1
    #print(f'App "{el.id}" Pricing Plans:', k)
    benefits = app_ref.document(el.id).collection('Key_Benefits').stream()
    for ben in benefits:
        m += 1
        
print('Apps Collection Total:', apps_num)        
print('Pricing_Plans Subcollection Total:', n)
print('Key_Benefits Subcollection Total:', m)

Apps Collection Total: 3547
Pricing_Plans Subcollection Total: 6275
Key_Benefits Subcollection Total: 9541


In [8]:
#Counting features
f_count = 0
for app_ref in db.collection('Apps').list_documents():
    for price_ref in app_ref.collection('Pricing_Plans').list_documents():
        f_count += price_ref.collection('Features').count().get()[0][0].value
print('Features Subcollection Total:', f_count)

Features Subcollection Total: 16270


#### 4b. Top 10 apps with the highest number of reviews

In [29]:
query = db.collection('Apps').order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
print('Top 10 apps with the highest number of reviews')
for result in results:
    app = result.to_dict()
    print(f'ID: {result.id} =>', 'Title: {}, Developer: {}, Rating: {}, Reviews_count: {}'.\
         format(app['app_title'], app['developer'], app['rating'], app['reviews_count']))


Top 10 apps with the highest number of reviews
ID: d9f142ee-b141-4dc4-9353-173db61d2eb0 => Title: Privy ‑ Exit Pop Ups & Email, Developer: Privy, Rating: 4.7, Reviews_count: 23078
ID: 78ea0810-c008-4a4e-a82f-de0c790e3286 => Title: Free Shipping Bar, Developer: Hextom, Rating: 4.9, Reviews_count: 8737
ID: b88488b0-9912-44d3-b736-224c36f09d95 => Title: Sales Pop ‑ Popup Notification, Developer: CartKit, Rating: 4.8, Reviews_count: 6905
ID: e528a60e-94f8-4e92-80e2-5bc6013b8283 => Title: BEST Currency Converter, Developer: Grizzly Apps, Rating: 4.8, Reviews_count: 5986
ID: be2640c4-01b5-4d52-9f68-cae8c0734d0d => Title: Recart FB Messenger Marketing, Developer: Recart, Rating: 4.8, Reviews_count: 5596
ID: 70bff9e0-4316-4cc6-84ce-92fcd1bc6925 => Title: EU Cookie Bar ‑ Cookie GDPR, Developer: Booster Apps, Rating: 4.7, Reviews_count: 5259
ID: 171816e2-27d4-4552-a65e-ab44a312fe04 => Title: Sales Pop Master ‑ Countdown, Developer: Autoketing, Rating: 4.8, Reviews_count: 4931
ID: 9025eff0-d714-4